## Importing necessary libraries

In [ ]:
import os
import numpy as np
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.utils import plot_model
from keras.applications.vgg19 import VGG19
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD
from keras.regularizers import l2
import matplotlib.pyplot as plt
import seaborn as sns

## Spliting the data folder into : Train and Test 

In [ ]:
# Set the path to your data directory
data_dir = "C:/Users/DELL/Desktop/datatest/D-E"

In [ ]:
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

## Define the model architecture

In [ ]:
# Define the input image size and number of classes
img_width = 1280
img_height = 720
batch_size = 32
num_classes = 2

In [ ]:
# Load the pre-trained VGG16 model
base_model = VGG19(
    weights='imagenet', # Load pre-trained weights from the ImageNet dataset
    include_top=False, # Set to False to exclude the top fully connected layers from the model
    input_shape=(img_width, img_height, 3) # The input shape of the images
)

In [ ]:
# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Build a new model on top of the pre-trained VGG16 model
model = Sequential()
model.add(base_model)
# flatten the output of the VGG19 base model
model.add(Flatten())
# add dense layers with dropout and L2 regularization
model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
# Print the model summary
model.summary()

In [ ]:
# plot the model architecture
plot_model(model, to_file='VGG19_model.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Compile the model
optimizer = SGD(lr=1e-4, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
# Define the data generators for image augmentation and rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255, # rescales the pixel values to the range [0, 1]
    rotation_range=20, # randomly rotates the images by up to 20 degrees
    width_shift_range=0.1, # randomly shifts the images horizontally by up to 10% of the image width
    height_shift_range=0.1, # randomly shifts the images vertically by up to 10% of the image height
    brightness_range=(0.8, 1.2), # randomly adjusts the brightness of the images by a factor between 0.8 and 1.2
    shear_range=0.2, # randomly applies shearing transformations to the images
    zoom_range=0.2, # randomly applies zooming transformations to the images
    horizontal_flip=True, # randomly flips the images horizontally
    fill_mode='nearest' #  fills in any empty pixels resulting from the transformations using the nearest neighboring pixel value
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Load the training and testing data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary"
)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary",
    shuffle=False
)

In [ ]:
# Define early stopping callback
early_stop = EarlyStopping(monitor=['val_loss', 'val_accuracy'], patience=10)

## Model training

In [ ]:
# Train the model
history = model.fit(
    train_data, 
    epochs=1000, 
    steps_per_epoch=train_data.n // train_data.batch_size,
    validation_data=test_data,
    validation_steps=test_data.n // test_data.batch_size,
    callbacks=[early_stop]
)

## Plot the training and validation loss and accuracy over the epochs

In [ ]:
# Set the Seaborn style
sns.set_style("darkgrid")
# Plot the training and validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Set the Seaborn style
sns.set_style("darkgrid")
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Save the accuracy plot to a file
plt.savefig('model_vgg19_accuracy.png')
# Save the accuracy plot to a file
plt.savefig('model_vgg19_loss.png')

## Confusion matrix

In [ ]:
# Predict the classes for the validation data
y_pred = model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get the true classes for the validation data
y_true = test_data.classes

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Plot the confusion matrix using seaborn
sns.heatmap(cm, annot=True, cmap='Blues')